# Visualization and Comparison

This notebook provides comprehensive visualizations comparing all models, including hybrid architectures, with focus on:
1. Model accuracy
2. Model size and efficiency
3. Optimization results
4. Performance trade-offs

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
plt.style.use('seaborn')
sns.set_palette('husl')

## 1. Load Results

In [ ]:
# Load all results
with open('results/training_results.pkl', 'rb') as f:
    training_results = pickle.load(f)

with open('results/optimization_results.pkl', 'rb') as f:
    optimization_results = pickle.load(f)

with open('results/regression_metrics.pkl', 'rb') as f:
    regression_metrics = pickle.load(f)

## 2. Model Performance Comparison

In [ ]:
def create_comparison_df():
    """Create DataFrame with model comparisons"""
    data = []
    
    for name, result in training_results.items():
        model_data = {
            'Model': name,
            'Accuracy': result['accuracy'],
            'Parameters': result['model'].count_params(),
            'Original Size (KB)': optimization_results[name]['Original']['size'],
            'Optimized Size (KB)': optimization_results[name]['Quantized']['size'],
            'Size Reduction (%)': ((optimization_results[name]['Original']['size'] - 
                                   optimization_results[name]['Quantized']['size']) / 
                                   optimization_results[name]['Original']['size'] * 100)
        }
        data.append(model_data)
    
    return pd.DataFrame(data)

comparison_df = create_comparison_df()
print("Model Comparison:")
print(comparison_df.to_string(index=False))

# Plot accuracy vs size
plt.figure(figsize=(12, 6))
plt.scatter(comparison_df['Optimized Size (KB)'], comparison_df['Accuracy'], 
           alpha=0.6, s=100)

for i, model in enumerate(comparison_df['Model']):
    plt.annotate(model, 
                 (comparison_df['Optimized Size (KB)'].iloc[i],
                  comparison_df['Accuracy'].iloc[i]))

plt.title('Model Accuracy vs Size (After Optimization)')
plt.xlabel('Model Size (KB)')
plt.ylabel('Accuracy')
plt.grid(True, alpha=0.3)
plt.show()

## 3. Optimization Impact Analysis

In [ ]:
# Plot optimization stages for each model
stages = ['Original', 'Architecture Optimized', 'Compressed', 'Quantized']
models = list(optimization_results.keys())

plt.figure(figsize=(15, 6))

# Size reduction plot
plt.subplot(1, 2, 1)
for model in models:
    sizes = [optimization_results[model][stage]['size'] for stage in stages]
    plt.plot(stages, sizes, marker='o', label=model)

plt.title('Model Size Through Optimization Stages')
plt.xlabel('Optimization Stage')
plt.ylabel('Size (KB)')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Accuracy preservation plot
plt.subplot(1, 2, 2)
for model in models:
    accuracies = [optimization_results[model][stage]['accuracy'] for stage in stages]
    plt.plot(stages, accuracies, marker='o', label=model)

plt.title('Model Accuracy Through Optimization Stages')
plt.xlabel('Optimization Stage')
plt.ylabel('Accuracy')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

## 4. Hybrid Model Analysis

In [ ]:
# Focus on hybrid models
hybrid_models = ['CNN-LSTM', 'CNN-BiLSTM']
original_models = ['1DCNN', 'LSTM', 'BiLSTM']

# Compare with original architectures
hybrid_comparison = comparison_df[
    comparison_df['Model'].isin(hybrid_models + original_models)
].copy()

# Plot comparison
plt.figure(figsize=(12, 6))

# Size vs Accuracy plot
plt.subplot(1, 2, 1)
for model_type, marker, size in zip(['Original', 'Hybrid'],
                                    ['o', 's'],
                                    [100, 150]):
    mask = hybrid_comparison['Model'].isin(hybrid_models if model_type == 'Hybrid' else original_models)
    plt.scatter(hybrid_comparison[mask]['Optimized Size (KB)'],
               hybrid_comparison[mask]['Accuracy'],
               label=model_type,
               marker=marker,
               s=size,
               alpha=0.6)

for i, row in hybrid_comparison.iterrows():
    plt.annotate(row['Model'],
                 (row['Optimized Size (KB)'], row['Accuracy']))

plt.title('Hybrid vs Original Models')
plt.xlabel('Optimized Size (KB)')
plt.ylabel('Accuracy')
plt.legend()

# Size reduction comparison
plt.subplot(1, 2, 2)
bars = plt.bar(hybrid_comparison['Model'],
               hybrid_comparison['Size Reduction (%)'])

# Color bars by model type
for i, bar in enumerate(bars):
    if hybrid_comparison['Model'].iloc[i] in hybrid_models:
        bar.set_color('orange')
    else:
        bar.set_color('blue')

plt.title('Size Reduction After Optimization')
plt.xlabel('Model')
plt.ylabel('Size Reduction (%)')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 5. Performance Trade-off Analysis

In [ ]:
def calculate_efficiency_score(row):
    """Calculate efficiency score (accuracy/size ratio)"""
    return row['Accuracy'] / row['Optimized Size (KB)'] * 1000  # Scale for readability

# Add efficiency score
comparison_df['Efficiency Score'] = comparison_df.apply(calculate_efficiency_score, axis=1)

# Plot efficiency scores
plt.figure(figsize=(10, 6))
bars = plt.bar(comparison_df['Model'], comparison_df['Efficiency Score'])

# Color bars by model type
for i, bar in enumerate(bars):
    if comparison_df['Model'].iloc[i] in hybrid_models:
        bar.set_color('orange')
    else:
        bar.set_color('blue')

plt.title('Model Efficiency (Accuracy/Size Ratio)')
plt.xlabel('Model')
plt.ylabel('Efficiency Score')
plt.xticks(rotation=45)

# Add value labels
for i, v in enumerate(comparison_df['Efficiency Score']):
    plt.text(i, v, f'{v:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Print summary
print("\nModel Efficiency Summary:")
print(comparison_df[['Model', 'Accuracy', 'Optimized Size (KB)', 'Efficiency Score']]
      .sort_values('Efficiency Score', ascending=False)
      .to_string(index=False))

## 6. Save Final Results

In [ ]:
# Save comparison results
comparison_df.to_csv('results/model_comparison.csv', index=False)

# Create and save final summary
final_summary = {
    'best_model': comparison_df.loc[comparison_df['Efficiency Score'].idxmax(), 'Model'],
    'hybrid_performance': {
        model: {
            'accuracy': comparison_df[comparison_df['Model'] == model]['Accuracy'].iloc[0],
            'size': comparison_df[comparison_df['Model'] == model]['Optimized Size (KB)'].iloc[0],
            'efficiency': comparison_df[comparison_df['Model'] == model]['Efficiency Score'].iloc[0]
        }
        for model in hybrid_models
    }
}

with open('results/final_summary.pkl', 'wb') as f:
    pickle.dump(final_summary, f)

print("\nResults saved successfully!")

# Print final recommendation
print("\nFinal Recommendation:")
print(f"Best performing model (efficiency-wise): {final_summary['best_model']}")
print("\nHybrid Model Performance:")
for model, metrics in final_summary['hybrid_performance'].items():
    print(f"\n{model}:")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Optimized Size: {metrics['size']:.2f} KB")
    print(f"Efficiency Score: {metrics['efficiency']:.2f}")